# Annotate TSS

Before building the base GRN, we need to annotate the coaccessible peaks and filter our active promoter/enhancer elements. First, we will identify the peaks around transcription starting sites (TSS). We will then merge the Cicero data with the TSS peak information and filter any peaks with weak connections to the TSS peaks. As such, the filtered peak data will only include TSS peaks and peaks with strong TSS connections. These will be our active promoter/enhancer elements for our base GRN.


Most of this code is from the CellOracle tutorial: https://morris-lab.github.io/CellOracle.documentation/notebooks/01_ATAC-seq_data_processing/option1_scATAC-seq_data_analysis_with_cicero/02_preprocess_peak_data.html

### 1. Import libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns


import os, sys, shutil, importlib, glob
from tqdm.notebook import tqdm

%config InlineBackend.figure_format = 'retina'

plt.rcParams['figure.figsize'] = [6, 4.5]
plt.rcParams["savefig.dpi"] = 300

In [2]:
import warnings
warnings.filterwarnings('ignore')
from celloracle import motif_analysis as ma
import celloracle as co
co.__version__

which: no R in (/apps/software/standard/core/jupyterlab/3.6.3-py3.11:/apps/software/standard/core/jupyterlab/3.6.3-py3.11/sbin:/apps/software/standard/core/jupyterlab/3.6.3-py3.11/bin:/home/apc5un/bedtools2/bin:/opt/mam/9.1.2/bin:/usr/local/bin:/usr/bin:/usr/local/sbin:/usr/sbin:/opt/slurm/current/bin:/opt/singularity/current/bin:/opt/rci/bin:/share/rci_apps/common/bin:/share/resources/HPCtools:/opt/mam/current/bin:/opt/apptainer/current/bin)


'0.14.0'

### 2. Load scATAC peak data and peak connection data made with Cicero

In this notebook, we will annotate and filter output from Cicero. Refer to data output from `make-cicero.R`.

Here, I will use either the pan-cardiac cicero connections, or cluster-by-cluster connections.

Below, I load the peaks and cicero CSV files. You'll do this for each timepoint and both WT/KO conditions.

In [4]:
timepoint = "E9"
wt_or_ko = "WT"
peaks = pd.read_csv(f"./data/base_grn_outputs/{timepoint}/{wt_or_ko}_peaks.csv", index_col=0)
peaks = peaks.x.values
peaks

array(['chr1_3057421_3057921', 'chr1_3191671_3192171',
       'chr1_3212387_3212887', ..., 'chrX_169899171_169899671',
       'chrX_169906174_169906674', 'chrX_169911120_169911620'],
      dtype=object)

In [12]:
cicero_connections = pd.read_csv(f"./data/base_grn_outputs/{timepoint}/{wt_or_ko}_cicero_connections.csv", index_col=0)
cicero_connections

,Peak1,Peak2,coaccess
1,chr10_100014083_100014583,chr10_99759099_99759599,0.000000
2,chr10_100014083_100014583,chr10_99824387_99824887,-0.029619
3,chr10_100014083_100014583,chr10_99841329_99841829,0.000000
4,chr10_100014083_100014583,chr10_99875647_99876147,0.000000
5,chr10_100014083_100014583,chr10_99882667_99883167,0.000526
...,...,...,...
22452771,chrX_99976238_99976738,chrX_99974318_99974818,-0.002235
22452772,chrX_99976238_99976738,chrX_99975210_99975710,-0.030297
22452774,chrX_99976238_99976738,chrX_100078706_100079206,-0.003441
22452775,chrX_99976238_99976738,chrX_100117571_100118071,-0.002384


### 3. Annotate transcription start sites

Use mm10 reference genome by setting ref_genome="mm10".

In [28]:
ma.SUPPORTED_REF_GENOME

,species,ref_genome,provider
0,Human,hg38,UCSC
1,Human,hg19,UCSC
2,Mouse,mm39,UCSC
3,Mouse,mm10,UCSC
4,Mouse,mm9,UCSC
5,S.cerevisiae,sacCer2,UCSC
6,S.cerevisiae,sacCer3,UCSC
7,Zebrafish,danRer7,UCSC
8,Zebrafish,danRer10,UCSC
9,Zebrafish,danRer11,UCSC


In [29]:
# There seems to be an error in the exporting of peaks from cicero, where 'chr19_24999500_25000000' was saved as 'chr19_24999500_2.5e+07', which causes an error
if timepoint == "E75":
    peaks[132381] = 'chr19_24999500_25000000'

tss_annotated = ma.get_tss_info(peak_str_list=peaks, ref_genome="mm10")

que bed peaks: 195967
tss peaks in que: 27046


The output file after the integration process has three columns: ["peak_id", "gene_short_name", "coaccess"].

“peak_id” is either the TSS peak or the peaks that have a connection to a TSS peak.

“gene_short_name” is the gene name that associated with the TSS site.

“coaccess” is the coaccessibility score between the peak and a TSS peak. If the score is 1, it means that the peak is a TSS itself.


In [30]:
tss_annotated.tail()

,chr,start,end,gene_short_name,strand
27041,chr5,119199809,119200309,Gm7538,-
27042,chr12,100899033,100899533,Gpr68,-
27043,chr4,129490949,129491449,Fam229a,-
27044,chr4,129491513,129492013,Fam229a,-
27045,chr17,24473696,24474196,Bricd5,+


### 4. Integrate TSS info and cicero connections
The output file after the integration process has three columns: ["peak_id", "gene_short_name", "coaccess"].

“peak_id” is either the TSS peak or the peaks that have a connection to a TSS peak.

“gene_short_name” is the gene name that associated with the TSS site.

“coaccess” is the coaccessibility score between the peak and a TSS peak. If the score is 1, it means that the peak is a TSS itself.

In [31]:
integrated = ma.integrate_tss_peak_with_cicero(tss_peak=tss_annotated,
                                               cicero_connections=cicero_connections)
print(integrated.shape)
integrated.head()

(913807, 3)


,peak_id,gene_short_name,coaccess
0,chr10_100015519_100016019,Cep290,0.004798
1,chr10_100015519_100016019,Kitl,1.000000
2,chr10_100015519_100016019,Tmtc3,0.056487
3,chr10_100016022_100016522,Cep290,0.008218
4,chr10_100016022_100016522,Kitl,0.285638


### 5. Filter peaks
Remove peaks with weak coaccessibility scores.

In [32]:
peak = integrated[integrated.coaccess >= 0.8]
peak = peak[["peak_id", "gene_short_name"]].reset_index(drop=True)

In [33]:
print(peak.shape)
peak.head()

(25922, 2)


,peak_id,gene_short_name
0,chr10_100015519_100016019,Kitl
1,chr10_100487279_100487779,Tmtc3
2,chr10_100487943_100488443,Cep290
3,chr10_100487943_100488443,Tmtc3
4,chr10_100589013_100589513,4930430F08Rik


### 6. Save data
Save the promoter/enhancer peaks.

In [34]:
peak.to_csv(f"./data/base_grn_outputs/{timepoint}/{wt_or_ko}_processed_peak_file.csv")